In [2]:
import requests as rq
import psycopg2 as p2
from datetime import datetime, timedelta
import pandas as pd
from pprint import pprint
import os
from dotenv import load_dotenv
from io import StringIO

##### Знакомство с API

In [3]:
# выясним, как работает API
api_url = 'http://final-project.simulative.ru/data'
params = {'date': 'yyyy-mm-dd'}
response = rq.get(api_url, params=params)
pprint(response.json())

[{'client_id': 534158,
  'discount_per_item': 25782,
  'gender': 'F',
  'price_per_item': 94995,
  'product_id': 29056,
  'purchase_datetime': 'yyyy-mm-dd',
  'purchase_time_as_seconds_from_midnight': 25288,
  'quantity': 64,
  'total_price': 4429632},
 {'client_id': 245757,
  'discount_per_item': 16549,
  'gender': 'F',
  'price_per_item': 65442,
  'product_id': 17469,
  'purchase_datetime': 'yyyy-mm-dd',
  'purchase_time_as_seconds_from_midnight': 5687,
  'quantity': 5,
  'total_price': 244465},
 {'client_id': 136773,
  'discount_per_item': 23201,
  'gender': 'F',
  'price_per_item': 23896,
  'product_id': 24265,
  'purchase_datetime': 'yyyy-mm-dd',
  'purchase_time_as_seconds_from_midnight': 81813,
  'quantity': 24,
  'total_price': 16680},
 {'client_id': 839569,
  'discount_per_item': 41281,
  'gender': 'M',
  'price_per_item': 72540,
  'product_id': 42641,
  'purchase_datetime': 'yyyy-mm-dd',
  'purchase_time_as_seconds_from_midnight': 31784,
  'quantity': 20,
  'total_price': 625

In [19]:
# видим, что API выдает данные с yyyy-mm-dd вместо даты
# посмотрим на длину полученного json
len(response.json())

3515

In [20]:
# теперь выполним точно такой же запрос несколько раз и посмотрим на длину json
for i in range(4):
    response = rq.get(api_url, params=params)
    print(len(response.json()))
# очевидно, перед нами генератор данных, который подставляет предоставленный параметр date  
# значит, цель - прописать скрипт, который сгенерирует данные за выбранный промежуток времени. Пусть он равняется 6 месяцам.

1161
4186
8746
1533


##### Заполнение базы историческими данными

In [21]:
# генерируем список интересующих нас дат
start_date = datetime.date(datetime.today() - timedelta(days=183))
end_date = datetime.date(datetime.today() - timedelta(days=1))

date_list = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# трансформируем в формат 'yyyy-mm-dd'
date_list_for_script = [f'{i.year}-{i.month:02d}-{i.day:02d}' for i in date_list]

In [4]:
# получаем данные для подключения к БД и настраиваем подключение
result = load_dotenv()
if not result:
    raise EnvironmentError('Не удалось загрузить файл .env или переменные окружения')

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASS')
db_port = os.getenv('DB_PORT')

conn = p2.connect(database=db_name, user=db_user, password=db_pass, host=db_host, port=db_port)

#conn.autocommit = True
cursor = conn.cursor()
conn.status

1

In [ ]:
# тест добавления инфы
data_for_db = []
response = rq.get(api_url, params={'date': date_list_for_script[0]})
data_for_db.extend(response.json())
data_for_db
# видим, что добавление 1 элемента списка занимает 0,6 секунды
# по идее формирование должно занять не более 110-115 секунд

[{'client_id': 243941,
  'gender': 'M',
  'purchase_datetime': '2025-08-27',
  'purchase_time_as_seconds_from_midnight': 68712,
  'product_id': 40952,
  'quantity': 23,
  'price_per_item': 45061,
  'discount_per_item': 3116,
  'total_price': 964735},
 {'client_id': 345647,
  'gender': 'M',
  'purchase_datetime': '2025-08-27',
  'purchase_time_as_seconds_from_midnight': 34049,
  'product_id': 24829,
  'quantity': 64,
  'price_per_item': 59244,
  'discount_per_item': 34787,
  'total_price': 1565248},
 {'client_id': 49270,
  'gender': 'F',
  'purchase_datetime': '2025-08-27',
  'purchase_time_as_seconds_from_midnight': 745,
  'product_id': 5993,
  'quantity': 30,
  'price_per_item': 73892,
  'discount_per_item': 8423,
  'total_price': 1964070},
 {'client_id': 795844,
  'gender': 'M',
  'purchase_datetime': '2025-08-27',
  'purchase_time_as_seconds_from_midnight': 52573,
  'product_id': 36758,
  'quantity': 13,
  'price_per_item': 65322,
  'discount_per_item': 32079,
  'total_price': 43215

In [ ]:
# формируем базу данных за полгода
data_for_db = []

for date in date_list_for_script:
    response = rq.get(api_url, params={'date': date})
    data_for_db.extend(response.json())

In [ ]:
# теперь задача - загрузить это в БД PostgreSQL
# создаем таблицу sales в БД, задаем типы данных
query = """create table if not exists sales (
    id serial primary key,
    client_id varchar(50),
    gender char(1),
    purchase_datetime timestamp,
    purchase_time_as_seconds_from_midnight integer,
    product_id varchar(50),
    quantity integer,
    price_per_item numeric(10, 2),
    discount_per_item numeric(10, 2),
    total_price numeric(12, 2)
)"""

try:
    cursor.execute(query)
    conn.commit()
except Exception as err:
    raise repr(err)

In [ ]:
# воспользуемся методом .copy_from для загрузки всех данных одним запросом к БД
upload_df = pd.DataFrame(data_for_db)

def upload_to_db(df, table_name):
    output = StringIO()
    df.to_csv(output, sep='\t', header=False, index=False)
    output.seek(0)
    columns = (
        'client_id', 'gender', 'purchase_datetime', 
        'purchase_time_as_seconds_from_midnight', 'product_id', 
        'quantity', 'price_per_item', 'discount_per_item', 'total_price'
    )
    cursor.copy_from(output, table_name, sep='\t', columns=columns)
    conn.commit()

upload_to_db(upload_df, 'sales')

In [ ]:
# проверим, успешно ли загрузились данные
query = """
    select client_id, purchase_datetime, total_price
    from sales
    order by purchase_datetime desc, total_price desc
    limit 5
"""
cursor.execute(query)
cursor.fetchall()
# данные загружены успешно

[('124074', datetime.datetime(2026, 2, 25, 0, 0), Decimal('8477954.00')),
 ('636275', datetime.datetime(2026, 2, 25, 0, 0), Decimal('8452854.00')),
 ('827944', datetime.datetime(2026, 2, 25, 0, 0), Decimal('7992635.00')),
 ('493270', datetime.datetime(2026, 2, 25, 0, 0), Decimal('7936288.00')),
 ('871390', datetime.datetime(2026, 2, 25, 0, 0), Decimal('7767770.00'))]

In [10]:
conn.close()

In [11]:
conn.closed

1